In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

In [74]:
pz14 = pd.read_csv('/home/rt2122/Data/scans_old/scans_extended/connected/ep14_thr0.1.csv')
pz40 = pd.read_csv('/home/rt2122/Data/scans_old/scans_extended/connected/ep40_thr0.1.csv')
pz_act10 = pd.read_csv(
    '/home/rt2122/Data/scans_old/scans_extended/connected/full_act_cut_ep10_thr0.1_step8.csv')
pz_act14 = pd.read_csv(
    '/home/rt2122/Data/scans_old/scans_extended/connected/full_act_cut_ep14_thr0.1_step8.csv')

In [75]:
det_cats = {'pz14' : pz14, 'pz40' : pz40, 'pz_act10' : pz_act10, 'pz_act14' : pz_act14}
for name in det_cats:
    det_cats[name] = det_cats[name][det_cats[name]['status'] != 'fn']
    det_cats[name].index = np.arange(len(det_cats[name]))

In [19]:
psz2 = None
with fits.open('/home/rt2122/Data/original_catalogs/psz2.fits') as hdul:
    data = hdul[1].data
    psz2 = Table(data).to_pandas()
mcxc = None
with fits.open('/home/rt2122/Data/original_catalogs/mcxc.fits') as hdul:
    data = hdul[1].data
    mcxc = Table(data).to_pandas()
rm = None
with fits.open('/home/rt2122/Data/original_catalogs/redmapper.fits.gz') as hdul:
    data = Table(hdul[1].data)
    names = [name for name in data.colnames if len(data[name].shape) <= 1]
    rm = data[names].to_pandas()
act = None
with fits.open('/home/rt2122/Data/original_catalogs/act.fits') as hdul:
    data = hdul[1].data
    act = Table(data).to_pandas()

In [21]:
true_cats = {'psz2' : psz2, 'mcxc' : mcxc, 'rm' : rm, 'act' : act}

In [29]:
true_cats['mcxc'].rename({'RAdeg' : 'RA', 'DEdeg' : 'DEC'}, axis='columns', inplace=True)
true_cats['act'].rename({'RADeg' : 'RA', 'decDeg' : 'DEC'}, axis='columns', inplace=True)

In [54]:
match_dist = 5 / 60

In [81]:
comp_df = []
recall_df = []
for det_name in det_cats:
    line = {}
    line_r = {}
    
    sc = SkyCoord(ra=np.array(det_cats[det_name]['RA'])*u.degree, 
                  dec=np.array(det_cats[det_name]['DEC'])*u.degree, frame='icrs')
    
    for tr_name in true_cats: 
        tr_sc = SkyCoord(ra=np.array(true_cats[tr_name]['RA'])*u.degree, 
                      dec=np.array(true_cats[tr_name]['DEC'])*u.degree, frame='icrs')
        idx, d2d, _ = sc.match_to_catalog_sky(tr_sc)
        matched = d2d.degree <= match_dist
        line[tr_name] = len(set((idx[matched])))
        line_r[tr_name] = line[tr_name] / len(true_cats[tr_name])
    line['fp'] = np.count_nonzero(det_cats[det_name]['status'] == 'fp')
    comp_df.append(pd.DataFrame(line, index=[det_name]))
    recall_df.append(pd.DataFrame(line_r, index=[det_name]))

line = {}
for tr_name in true_cats:
    line[tr_name] = len(true_cats[tr_name])
line['fp'] = 0
comp_df.append(pd.DataFrame(line, index=['all']))
comp_df = pd.concat(comp_df)
recall_df = pd.concat(recall_df)

In [82]:
comp_df

,psz2,mcxc,rm,act,fp
pz14,1486,721,1241,847,15828
pz40,1501,736,1273,870,17306
pz_act10,1359,657,1029,925,16316
pz_act14,1361,667,1208,1281,16484
all,1653,1743,26111,4195,0


In [83]:
recall_df

,psz2,mcxc,rm,act
pz14,0.898972,0.413655,0.047528,0.201907
pz40,0.908046,0.422260,0.048753,0.207390
pz_act10,0.822142,0.376936,0.039409,0.220501
pz_act14,0.823351,0.382674,0.046264,0.305364
